<a href="https://colab.research.google.com/github/NinadShegokar/Langchain/blob/main/Reduce_SUMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain scikit-learn langchain_community tiktoken langchain-openai langchainhub chromadb  sentence-transformers   llama-cpp-python
!pip install pypdf dspy-ai fpdf

from IPython.display import clear_output
clear_output()


In [ ]:
#imports
import langchain
from torch import cuda, bfloat16
from fpdf import FPDF
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader,PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain,ConversationalRetrievalChain,StuffDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import CTransformers
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, LLMChain

In [ ]:
#load the pdf files from the path
loader = PyPDFLoader(r'/content/Schizophrenia.pdf')
docs = loader.load()


In [ ]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=100
    )
split_docs = text_splitter.split_documents(docs)


In [ ]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy


In [ ]:
!wget "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_K_M.gguf"

--2024-04-06 18:02:09--  https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 65.8.178.93, 65.8.178.27, 65.8.178.12, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.93|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/fe/17/fe17596731f84a0d03bece77489780bc7e068323c0aeca88b6393d3e9e65dd49/503580dce392c6e64669ad21a77023ba2a17baa0c381250fb67c11ba6406a85e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27zephyr-7b-beta.Q4_K_M.gguf%3B+filename%3D%22zephyr-7b-beta.Q4_K_M.gguf%22%3B&Expires=1712685729&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMjY4NTcyOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ZlLzE3L2ZlMTc1OTY3MzFmODRhMGQwM2JlY2U3NzQ4OTc4MGJjN2UwNjgzMjNjMGFlY2E4OGI2MzkzZDNlOWU2NWRkNDkvNTAzNTgwZGNlMzkyYzZlNjQ2NjlhZDIxYTc3MDIzYmEyYTE

In [ ]:
llm = LlamaCpp(
    model_path="/content/zephyr-7b-beta.Q4_K_M.gguf",
    temperature=0.75,
    max_tokens=1000,
    top_p=1,
    n_ctx=35000,
    verbose=True,  # Verbose is required to pass to the callback manager
)
EMBEDDING_MODEL_NAME = "thenlper/gte-small"

embd = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/zephyr-7b-beta.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-beta
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.at

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
map_prompt = """
Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["docs"])

In [ ]:
combine_prompt = """
Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```{text}```
BULLET POINT SUMMARY:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["docs"])

In [ ]:
summary_chain = load_summarize_chain(llm=llm,
                                     chain_type='map_reduce',
                                     map_prompt=map_prompt_template,
                                     combine_prompt=combine_prompt_template,
                                     verbose=True
                                    )

In [ ]:
output = summary_chain.run(docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise summary of the following:
"Schizophrenia is a mental disorder characterized by reoccurring episodes of psychosis, including 
hallucinations (typically hearing voices), delusions (i.e., paranoia), disorganized thinking, social 
withdrawal, and flat affect. Symptoms develop gradually, t ypically beginning during young adulthood 
and persisting indefinitely. Diagnosis is based on observed behaviour and psychiatric history, 
requiring symptoms to be present for at least six months (according to the DSM -5) or one month 
(according to the ICD -11). Many individuals with schizophrenia have co -occurring mental disorders, 
such as substance use disorders, depressive disorders, anxiety disorders, and obsessive -compulsive 
disorder.  
Approximately 0.3% to 0.7% of people are diagnosed with schizophrenia du ring their lifetime. Males 
are more commonly affected and typi

Llama.generate: prefix-match hit

llama_print_timings:        load time =    5375.85 ms
llama_print_timings:      sample time =     161.06 ms /   234 runs   (    0.69 ms per token,  1452.90 tokens per second)
llama_print_timings: prompt eval time =  532930.22 ms /   931 tokens (  572.43 ms per token,     1.75 tokens per second)
llama_print_timings:        eval time =  191783.95 ms /   233 runs   (  823.11 ms per token,     1.21 tokens per second)
llama_print_timings:       total time =  726497.62 ms /  1164 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    5375.85 ms
llama_print_timings:      sample time =     147.71 ms /   220 runs   (    0.67 ms per token,  1489.39 tokens per second)
llama_print_timings: prompt eval time =  422152.52 ms /   783 tokens (  539.15 ms per token,     1.85 tokens per second)
llama_print_timings:        eval time =  187354.55 ms /   219 runs   (  855.50 ms per token,     1.17 tokens per second)
llama_print_timings:       to


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```Schizophrenia is a chronic mental disorder characterized by recurrent episodes of psychosis including hallucinations, delusions, disorganized thinking, social withdrawal, and flat affect. Symptoms typically develop during young adulthood and persist indefinitely. Diagnosis requires symptoms to be present for at least six months (DSM-5) or one month (ICD-11). Co-occurring mental disorders are common. Approximately 0.3% to 0.7% of people develop schizophrenia during their lifetime, with males more commonly affected and an earlier onset than females. Causes include genetic and environmental factors such as cannabis use during adolescence, infections, parental age, and prenatal nutrition. Treatment typically involves antipsychotic medication, counseling,


llama_print_timings:        load time =    5375.85 ms
llama_print_timings:      sample time =     319.41 ms /   471 runs   (    0.68 ms per token,  1474.62 tokens per second)
llama_print_timings: prompt eval time =  264191.46 ms /   496 tokens (  532.64 ms per token,     1.88 tokens per second)
llama_print_timings:        eval time =  380542.83 ms /   471 runs   (  807.95 ms per token,     1.24 tokens per second)
llama_print_timings:       total time =  647912.46 ms /   967 tokens



> Finished chain.

> Finished chain.


In [ ]:
print (output)

1. Schizophrenia is a chronic mental disorder characterized by recurrent episodes of psychosis including hallucinations, delusions, disorganized thinking, social withdrawal, and flat affect.
2. Symptoms typically develop during young adulthood and persist indefinitely. Diagnosis requires symptoms to be present for at least six months (DSM-5) or one month (ICD-11).
3. Co-occurring mental disorders are common, with approximately 0.3% to 0.7% of people developing schizophrenia during their lifetime. Males more commonly affected and an earlier onset than females.
4. Causes include genetic and environmental factors such as cannabis use during adolescence, infections, parental age, and prenatal nutrition.
5. Treatment typically involves antipsychotic medication, counseling, job training, and social rehabilitation.
6. Remission of symptoms is possible for some individuals, while others experience worsening or remitting episodes over time.
7. Schizophrenia affects approximately 1 in 300 people